---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re


/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows',100)

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [10]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open('university_towns.txt') as file:
        df_list = []
        town_list = [i.rstrip() for i in file]
        for i in town_list:
            if '[edit]' in i:
                temp = (re.sub(r'\[.*\]', '',i)).strip()
            else:
                df_list.append([temp,(re.sub(r'\(.*\)|\[.*\]', '',i)).strip()])
    univer_twn = pd.DataFrame(df_list,columns = ['State','RegionName'])

    
    return univer_twn
get_list_of_university_towns()


# def get_list_of_university_towns():

#     with open("university_towns.txt") as f:
#         townslist = f.readlines()
    
#     townslist = [x.rstrip() for x in townslist]
    
#     townslist2 = list()
    
#     for i in townslist:
#         if i[-6:] == '[edit]':
#             temp_state = i[:-6]
#         elif '(' in i:
#             town = i[:i.index('(') - 1]
#             townslist2.append([temp_state, town])
#         else:
#             town = i
#             townslist2.append([temp_state, town])

#     collegedf = pd.DataFrame(townslist2, columns=['State','RegionName'])


#     return collegedf

# get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    cols = (['Year',"Annual_GDP","Annual_GDP 2009",'Year_Q',"Quaterly_GDP","Quaterly_GDP 2009"])
    gdp = pd.read_excel('gdplev.xls',header = None, skiprows = 8)
    gdp.dropna(axis=1, how='all',inplace=True)
    gdp.columns=cols
    gdp.drop(['Year','Annual_GDP 2009','Annual_GDP','Quaterly_GDP'],axis=1,inplace=True)
    gdp = gdp[gdp['Year_Q']>= '2000q1']
    gdp.reset_index(inplace=True)
    for i in range(0,len(gdp)):
        if (gdp.iloc[i-1][2] < gdp.iloc[i-2][2]) and (gdp.iloc[i][2] < gdp.iloc[i-1][2]):
            start_ress = gdp.iloc[i-3][1]
    return start_ress
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    cols = (['Year',"Annual_GDP","Annual_GDP 2009",'Year_Q',"Quaterly_GDP","Quaterly_GDP 2009"])
    gdp = pd.read_excel('gdplev.xls',header = None, skiprows = 8)
    gdp.dropna(axis=1, how='all',inplace=True)
    gdp.columns=cols
    gdp.drop(['Year','Annual_GDP 2009','Annual_GDP','Quaterly_GDP'],axis=1,inplace=True)
    gdp = gdp[gdp['Year_Q']>= '2000q1']
    gdp.reset_index(inplace=True)
    ress_start = gdp[gdp['Year_Q'] >= get_recession_start()]
    for i in range(0,len(ress_start)):
        if (ress_start.iloc[i-1][2] > ress_start.iloc[i-2][2]) and (ress_start.iloc[i][2] > ress_start.iloc[i-1][2]):
            return ress_start.iloc[i][1]
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    cols = (['Year',"Annual_GDP","Annual_GDP 2009",'Year_Q',"Quaterly_GDP","Quaterly_GDP 2009"])
    gdp = pd.read_excel('gdplev.xls',header = None, skiprows = 8)
    gdp.dropna(axis=1, how='all',inplace=True)
    gdp.columns=cols
    gdp.drop(['Year','Annual_GDP 2009','Annual_GDP','Quaterly_GDP'],axis=1,inplace=True)
    gdp = gdp[gdp['Year_Q']>= '2000q1']
    gdp.reset_index(inplace=True)
    ress_start = gdp[gdp['Year_Q'] >= get_recession_start()]

    return ress_start[ress_start['Quaterly_GDP 2009']== min(ress_start['Quaterly_GDP 2009'])].iloc[:,1].tolist()[0]
get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    City_Zhvi = pd.read_csv('City_Zhvi_AllHomes.csv')
    City_Zhvi_r = pd.concat([City_Zhvi.loc[:, 'RegionName':'State'],City_Zhvi.loc[:, '2000-01':]], axis = 1)
    City_Zhvi_r['State'].replace(states, inplace= True)
    for i in range(2,City_Zhvi_r.shape[1]):
        if City_Zhvi_r.columns[i][-3:]=='-01':
            City_Zhvi_r.rename(columns={City_Zhvi_r.columns[i]:City_Zhvi_r.columns[i][:4]+'q1',City_Zhvi_r.columns[i+1]:City_Zhvi_r.columns[i+1][:4]+'q1',City_Zhvi_r.columns[i+2]:City_Zhvi_r.columns[i+2][:4]+'q1'}, inplace=True)
        if City_Zhvi_r.columns[i][-3:]=='-04':
            City_Zhvi_r.rename(columns={City_Zhvi_r.columns[i]:City_Zhvi_r.columns[i][:4]+'q2',City_Zhvi_r.columns[i+1]:City_Zhvi_r.columns[i+1][:4]+'q2',City_Zhvi_r.columns[i+2]:City_Zhvi_r.columns[i+2][:4]+'q2'}, inplace=True)
        if City_Zhvi_r.columns[i][-3:]=='-07' and City_Zhvi_r.columns[i+1][-5:] != '16-08' :
            City_Zhvi_r.rename(columns={City_Zhvi_r.columns[i]:City_Zhvi_r.columns[i][:4]+'q3',City_Zhvi_r.columns[i+1]:City_Zhvi_r.columns[i+1][:4]+'q3',City_Zhvi_r.columns[i+2]:City_Zhvi_r.columns[i+2][:4]+'q3'}, inplace=True)
        if City_Zhvi_r.columns[i][-3:]=='-07':
            City_Zhvi_r.rename(columns={City_Zhvi_r.columns[i]:City_Zhvi_r.columns[i][:4]+'q3',City_Zhvi_r.columns[i+1]:City_Zhvi_r.columns[i+1][:4]+'q3'}, inplace=True)
        if City_Zhvi_r.columns[i][-3:]=='-10':
            City_Zhvi_r.rename(columns={City_Zhvi_r.columns[i]:City_Zhvi_r.columns[i][:4]+'q4',City_Zhvi_r.columns[i+1]:City_Zhvi_r.columns[i+1][:4]+'q4',City_Zhvi_r.columns[i+2]:City_Zhvi_r.columns[i+2][:4]+'q4'}, inplace=True)

     

    City_Zhvi_r.set_index(['State','RegionName'], inplace= True)

    City_Zhvi_r = City_Zhvi_r.groupby(by=City_Zhvi_r.columns, axis=1).mean()
    City_Zhvi_r.head()
    City_Zhvi_r = City_Zhvi_r.sort_index()
    return City_Zhvi_r
convert_housing_data_to_quarters()


2000q1         2000q2  \
State     RegionName                                              
Alabama   Adamsville                69033.333333   69166.666667   
          Alabaster                122133.333333  123066.666667   
          Albertville               73966.666667   72600.000000   
          Arab                      83766.666667   81566.666667   
          Ardmore                            NaN            NaN   
          Axis                               NaN            NaN   
          Baileyton                          NaN            NaN   
          Bay Minette               81700.000000   78533.333333   
          Bayou La Batre            44066.666667   44500.000000   
          Bessemer                           NaN            NaN   
          Birmingham                54033.333333   54400.000000   
          Boaz                      70866.666667   70266.666667   
          Brent                     92933.333333   94333.333333   
          Brighton                           NaN            NaN   
          Brookwood                 92566.666667   95100.000000   
          Buhl                      90800.000000   94600.000000   
          Calera                   108933.333333  110366.666667   
          Center Point              80966.666667   81233.333333   
          Centreville               95300.000000   96566.666667   
          Chalkville                94100.000000   94433.333333   
          Chancellor                         NaN            NaN   
          Chelsea                  162066.666667  167033.333333   
          Chickasaw                 51200.000000   53666.666667   
          Chunchula                 80266.666667   81766.666667   
          Citronelle                64833.333333   66633.333333   
          Clay                     120900.000000  122266.666667   
          Coden                     62600.000000   64800.000000   
          Coker                    118100.000000  120766.666667   
          Concord                   78600.000000   78700.000000   
          Cottondale               100833.333333  102633.333333   
          Crane Hill                         NaN            NaN   
          Creola                    77066.666667   78233.333333   
          Cullman                            NaN            NaN   
          Daphne                   123166.666667  120233.333333   
          Dauphin Island           177700.000000  176466.666667   
          Deatsville               112733.333333  113500.000000   
          Decatur                            NaN            NaN   
          Duncanville              108100.000000  112033.333333   
          Edgewater                          NaN            NaN   
          Eight Mile                69000.000000   69033.333333   
          Elberta                  134700.000000  147033.333333   
          Elmore                             NaN            NaN   
          Fairfield                 59333.333333   59100.000000   
          Fairhope                 161400.000000  161933.333333   
          Foley                    100233.333333  106100.000000   
          Forestdale                88966.666667   89500.000000   
          Fosters                  107500.000000  110200.000000   
          Fultondale                90066.666667   88366.666667   
          Gardendale               114200.000000  114366.666667   
          Good Hope                          NaN            NaN   
...                                          ...            ...   
Wisconsin Town of Sheboygan         91533.333333   92600.000000   
          Town of Sheboygan Falls  104466.666667  107100.000000   
          Town of Sun Prairie      152933.333333  158133.333333   
          Town of Verona           190600.000000  194466.666667   
          Town of Walworth          93200.000000   86200.000000   
          Town of West Bend        127466.666667  127233.333333   
          Town of Whitewater        94233.333333   88500.000000   
          Town of Wind

In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    university_town_list = get_list_of_university_towns()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    housing = convert_housing_data_to_quarters()
    
    housing = housing.loc[:,rec_start:rec_bottom]
    
    housing.reset_index(inplace=True)
    university_town_list.reset_index(inplace=True)
    housing['reces_diff'] = housing[rec_start] - housing[rec_bottom]

    university_town = pd.merge(housing, university_town_list, on=['State', 'RegionName'])
    
    university_town['Flag'] = True
    housing = pd.merge(housing, university_town, how='outer', on = ['State', 'RegionName',
                                                              rec_bottom, rec_start, 
                                                              'reces_diff'])
    
    housing['Flag'] = housing['Flag'].fillna(False)
    
    university = housing[housing['Flag'] == True]
    
    non_university = housing[housing['Flag'] == False]
    
    t, p = ttest_ind(university['reces_diff'].dropna(),non_university['reces_diff'].dropna())
    different =  True if p<0.01 else False
    better = 'university town' if university['reces_diff'].mean() < non_university['reces_diff'].mean() else 'non-university town'
    
    return different, p, better
run_ttest()

(True, 0.002582640431066486, 'university town')